# Bounding Type II error for the generalized signature kernel MMD with examples (Gaussian processes)

## Z Issa May 2023

In this notebook, we provide code and worked examples for studying the Type II error present when calculating unbiased estimates for the signature kernel MMD (for fixed sample size $N \in \mathbb{N}$) between two sets of paths.

## 0. Imports

In [ ]:
import math

import torch
import torchsde
import sigkernel
import iisignature
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from src.utils.helper_functions.plot_helper_functions import make_grid, golden_dimensions

In [ ]:
is_cuda = torch.cuda.is_available()
device = 'cuda' if is_cuda else 'cpu'

if not is_cuda:
    print("Warning: CUDA not available; falling back to CPU but this is likely to be very slow.")
    
# You realistically need GPU access (either natively or via cloud computing) to run this notebook.

## 1. Generate data

In [ ]:
# Build some gbm paths
class GeometricBrownianMotion(torch.nn.Module):
    
    def __init__(self, mu, sigma, noise_type: str, sde_type: str):
        super().__init__()
        self.mu      = torch.nn.Parameter(torch.tensor(mu, dtype=torch.float64), requires_grad=True) 
        self.sigma   = torch.nn.Parameter(torch.tensor(sigma, dtype=torch.float64), requires_grad=True)
        self.noise_type = noise_type
        self.sde_type   = sde_type
        
    def f(self, t, y):
        return self.mu*y
    
    def g(self, t, y):
        return self.sigma*y

class BrownianMotionDrift(torch.nn.Module):
    
    def __init__(self, mu, sigma, noise_type: str, sde_type: str):
        super().__init__()
        self.mu         = torch.nn.Parameter(torch.tensor(mu, dtype=torch.float64), requires_grad=True) 
        self.sigma      = torch.nn.Parameter(torch.tensor(sigma, dtype=torch.float64), requires_grad=True)
        self.noise_type = noise_type
        self.sde_type   = sde_type
        
    def f(self, t, y):
        return torch.zeros(size=y.size()).to(y.device) + self.mu
    
    def g(self, t, y):
        return torch.zeros(size=y.size()).to(y.device) + self.sigma

In [ ]:
# sde params
mu0, sig = 0., 0.2
mu1, beta = 0., 0.3
noise_type = "diagonal"
sde_type   = "ito"

# Grid params
T           = 1
grid_points = 32
dt_scale    = 1e-1  # Finer refinements give better solutions (but slower)
ts          = torch.linspace(0, T, grid_points).to(device)

# Path bank params

batch_size = 32768
state_size = 1

h0_model = BrownianMotionDrift(mu0, sig, noise_type, sde_type).to(device)
h1_model = BrownianMotionDrift(mu1, beta, noise_type, sde_type).to(device)

y0 = torch.full(size=(batch_size, state_size), fill_value=0.).to(device)

_dt = dt_scale*torch.diff(ts)[0]

with torch.no_grad():
    h0_paths = torchsde.sdeint(h0_model, y0, ts, method='euler', dt = _dt)
    h1_paths = torchsde.sdeint(h1_model, y0, ts, method='euler', dt = _dt)

In [ ]:
h0_paths = torch.cat([ts.unsqueeze(-1).expand(batch_size, ts.size(0), 1), torch.transpose(h0_paths, 1, 0)], dim=2)
h1_paths = torch.cat([ts.unsqueeze(-1).expand(batch_size, ts.size(0), 1), torch.transpose(h1_paths, 1, 0)], dim=2)

In [ ]:
# Quick plot to verify 
n_plot_paths = 32 
label_first = True

for p0, p1 in zip(h0_paths[:n_plot_paths].cpu(), h1_paths[:n_plot_paths].cpu()):
    plt.plot(ts.cpu(), p0[:, 1], color="dodgerblue", alpha=0.5, label="$H_0$" if label_first else "")
    plt.plot(ts.cpu(), p1[:, 1], color="tomato", alpha=0.5, label="$H_1$" if label_first else "")
    label_first = False
plt.legend()
make_grid()
plt.title(fr"$H_0$ vs $H_1$, scaled BM, $\sigma = {sig:.2f}$, $\beta = {beta:.2f}$");

## 3. Sample $H_0$ and $H_1$ MMD distributions, define Type II error function

In [ ]:
def return_mmd_distributions(h0_paths, h1_paths, estimator, n_atoms=128, n_paths=32):

    h0_dists = torch.zeros(n_atoms)
    h1_dists = torch.zeros(n_atoms)

    rand_ints = torch.randint(0, batch_size, size=(n_atoms, 3, n_paths))

    with torch.no_grad():
        for i, ii in enumerate(tqdm(rand_ints)):
            x1, x2 = h0_paths[ii[0]], h0_paths[ii[1]]
            y = h1_paths[ii[-1]]

            h0_dists[i] = estimator(x1, x2)
            h1_dists[i] = estimator(x1, y)
            
    return h0_dists, h1_dists

def expected_type2_error(dist, crit_value):
    n_atoms = dist.shape[0]
    num_fail = dist <= crit_value
    return sum(num_fail)/n_atoms

In [ ]:
def increment_transform(path: torch.tensor) -> torch.tensor:
    """
    Returns the path of absolute increments Y associated to a path X. This is given by Y_0 = X_0 and
    Y_i = Y_{i-1} + |X_i - X_{i-1}|.

    :param path:    Path to calculate increment transform of.
    :return:        Transformed path.
    """
    device = path.device
    N, l, d = path.shape
    
    res = torch.zeros(path.shape).to(device)
    res[..., 0] = path[..., 0]
    res[..., 1:, 1:] = torch.abs(path[..., 1:].diff(axis=1)).cumsum(dim=1)
    
    res[..., 1:] += torch.tile(path[:, 0, 1:].unsqueeze(-1), (1, l, 1))

    return res

def scale_transform(path: torch.tensor, scaler: torch.float32) -> torch.tensor:
    device = path.device
    res    = torch.zeros(path.shape).to(device)
    
    scaler_ = torch.tensor(scaler).to(device)
    
    res[..., 1:] = path[..., 1:]*scaler_
    
    return res

## 4. Initialize signature kernel

In [ ]:
# We work with the linear kernel (for now), a la the paper. 

dyadic_order  = 1  # At least 1 for accurate calculation of the signature kernel
static_kernel = sigkernel.LinearKernel()

signature_kernel = sigkernel.SigKernel(static_kernel=static_kernel, dyadic_order=dyadic_order)

In [ ]:
n_atoms   = 1024
n_paths   = 64
max_batch = 64
_scale_f  = torch.tensor(n_paths).sqrt()
alpha     = 0.05
time_normalisation = True
_scaler = 1 

t_h0_paths = scale_transform(h0_paths.clone(), _scaler)
t_h1_paths = scale_transform(h1_paths.clone(), _scaler)
estimator = lambda x, y: signature_kernel.compute_mmd(x, y, max_batch=max_batch)

if time_normalisation:
    t_h0_paths[..., 0] /= _scaler*T
    t_h1_paths[..., 0] /= _scaler*T

h0_dists, h1_dists = return_mmd_distributions(
    t_h0_paths, 
    t_h1_paths, 
    estimator,
    n_atoms=n_atoms, 
    n_paths=n_paths
)

In [ ]:
crit_val = h0_dists.sort()[0][int(n_atoms*(1-alpha))]

plt.figure(figsize=golden_dimensions(4))
n_bins = int(n_atoms/8)
plt.title(f"{n_paths} paths, {n_atoms} atoms. Expected Type II error: {100*expected_type2_error(h1_dists, crit_val):.2f}%", fontsize="small")
plt.hist(sorted(h0_dists)[16:-16], bins=n_bins, color="dodgerblue", alpha=0.5, label="$H_0$", density=True)
plt.hist(sorted(h1_dists)[16:-16], bins=n_bins, color="tomato"    , alpha=0.5, label="$H_1$", density=True)
plt.legend()
make_grid()

## 5. Get level-$k$ signature term contributions

Suppose $\mathcal{X}$ is a compact subset of $\mathcal{C}_p([0, T]; V)$, the set of unparametrized, tree-reduced paths with finite $p$-variation for $1 \le p < 2$ (i.e., the set of equivalence classes of paths of finite $p$-variation where the equivalence relation $\sim_\tau$ denotes tree-like equivalence). Let $S: \mathcal{X} \to T((V))$ denote the signature mapping and $S(X)^k$ the terms corresponding to the $k^{\text{th}}$ level signature.

Let $\mathcal{D}_{k_\phi}: \mathcal{P}(\mathcal{X}) \times \mathcal{P}(\mathcal{X}) \to \mathbb{R}$ denote the maximum mean discrepancy with the $\phi$-siganture kernel. Because we know that

$$ k_\phi(X, Y) = \sum_{k\ge 0}\phi(k) \langle S(X)^k, S(Y)^k \rangle_k, $$

where $\langle \cdot, \cdot \rangle_k = \langle \cdot, \cdot \rangle_{V^{\otimes k}}$ and thus $\langle A, B \rangle_{T((V))} = \sum_{k\ge 0} \langle A^k, B^k \rangle_k$, the Hilbert-Schmidt norm. Then we know that 

$$\mathcal{D}_{k_\phi}(\mathbb{P}, \mathbb{Q})^2 = \mathbb{E}_{\mathbb{P}}[k_\phi(X, X')] - 2\mathbb{E}_{\mathbb{P}, \mathbb{Q}}[k_\phi(X, Y)] + \mathbb{E}_{\mathbb{Q}}[k_\phi(Y, Y')]$$

and thus 

$$\mathcal{D}_{k_\phi}(\mathbb{P}, \mathbb{Q})^2 = \phi(0) + \sum_{k\ge 1}\phi(k) \left[\mathbb{E}\Lambda_k(\mathbb{P}, \mathbb{P}) - 2\mathbb{E}\Lambda_k(\mathbb{P}, \mathbb{Q}) + \mathbb{E}\Lambda_k(\mathbb{Q}, \mathbb{Q}) \right]$$

where $\mathbb{E}\Lambda_k: \mathcal{P}(\mathcal{X}) \times \mathcal{P}(\mathcal{X}) \to \mathbb{R}$ is defined by $$\mathbb{E}\Lambda_k(\mathbb{P}, \mathbb{Q}) = \sum_{i_1,\dots,i_k \in \mathcal{W}(V^{\otimes k})} \mathbb{E}_\mathbb{P}[S(X)^{i_1,\dots,i_k}]\mathbb{E}_\mathbb{Q}[S(Y)^{i_1,\dots,i_k}].$$

This means we can view the (squared) MMD as the sum of (weighted) level contributions. Let's reconstruct these now for this example.

<b>Remark</b>: Here we can explicitly evaluate the $\phi$-function $\phi: \mathbb{N} \cup \{0\} \to \mathbb{R}$. In practice, you cannot, as one does not calculate terms directly but rather solves a PDE to calculate the signature kernel. However, if a desired $\phi$ solves the Hamburger moment problem, $$\phi(k) = \int_{\Omega} \pi^k \,d\mu(\pi)$$ for a probability measure $\mu \in \mathcal{P}(\mathbb{R})$, then we can calculate the $\phi$-kernel as a weighted sum of $k_{\lambda}$-kernels (where $\lambda \sim \mu$).  

In [ ]:
def get_signatures(paths, k):
    # Calculate the signature up to the required order
    
    n, l, d = paths.shape
        
    if k == 0:
        return torch.zeros(n) + 1.
    
    if type(paths) == torch.Tensor:
        paths = np.array(paths.cpu().detach())
        
    sigs  = iisignature.sig(paths, k)
    return torch.tensor(sigs)

def get_level_k_signatures(paths, k):
    # Calculate the signature up to the required order
    _, _, d= paths.shape
    
    sigs  = get_signatures(paths, k)
    stloc = sum((d**ki for ki in range(k))) - 1
    
    return sigs[:, stloc:]

def get_level_k_signatures_from_signatures(signatures, k, d):
    """
    Extracts the level k terms from the bank of signatures directly. Requires more data in order to get the correct
    indexing

    :param signatures:      Bank of signatures
    :param k:               Order of level terms to extract
    :param d:               Dimension of path
    :return:                Level k signature terms
    """

    stloc = sum((d ** ki for ki in range(k))) - 1
    endloc = sum((d ** ki for ki in range(k+1))) - 1

    return signatures[:, stloc:endloc]

For batched samples $X=\{X_i\}_{i=1}^N, Y=\{Y_i\}_{i=1}^N$, the (unbiased) MMD estimator can be re-written as 

$$ D_{k_\phi}^u(X, Y)^2 = \sum_{k\ge 0} \left(\frac{1}{N(N-1)}\sum_{i\ne j}\phi(k)\left[\Lambda_k(X_i, X_j) + \Lambda_k(Y_i, Y_j)\right] - \frac{2}{N^2}\sum_{i,j=1}^N \phi(k)\Lambda_k(X_i, Y_j)\right),$$

where $\Lambda_k(X, Y) = \mathbb{E}\Lambda_k(\delta_X, \delta_Y)$. Thus the level-$k$ contribution is given by 

$$M_k(X, Y) = \frac{1}{N(N-1)}\sum_{i\ne j}\phi(k)\left[\Lambda_k(X_i, X_j) + \Lambda_k(Y_i, Y_j)\right] - \frac{2}{N^2}\sum_{i,j=1}^N \phi(k)\Lambda_k(X_i, Y_j).$$

Written in matrix form, 

$$M_k(X, Y) = \phi(k) \left(\frac{1}{N(N-1)}(\mathbb{E}[G^u_{XX}] + \mathbb{E}[G^u_{YY}]) - \frac{2}{N^2}\mathbb{E}[G_{XY}]\right), $$

where $G_{XY}$ is the Gram matrix between $X, Y$, superscript $u$ denotes unbiased (diagonals removed).

In [ ]:
def Lambda_k(X, Y, k):
    """Calculates lambda_k given two sets of paths, shape N x l x d"""
    
    if k == 0:
        return 1.
    
    sigX = torch.mean(get_level_k_signatures(X, k), dim=0)
    sigY = torch.mean(get_level_k_signatures(Y, k), dim=0)
    
    return torch.dot(sigX, sigY)

def Gramda_k(X, Y, k):
    N = X.size(0)
    
    sigsX = get_level_k_signatures(X, k)
    sigsY = get_level_k_signatures(Y, k)
    
    return torch.einsum("ik,jk->ij", sigsX, sigsY)

def M_k(X, Y, k, phik=lambda x: 1):
    if k == 0:
        return 0.
    
    N = X.size(0)
    
    gXX = Gramda_k(X, X, k)
    gYY = Gramda_k(Y, Y, k)
    gXY = Gramda_k(X, Y, k)

    gXX -= torch.diag(torch.diag(gXX))
    gYY -= torch.diag(torch.diag(gYY))

    
    res1 = (torch.sum(gXX) + torch.sum(gYY))/(N*(N-1))
    res2 = 2*torch.sum(gXY)/(N**2)
    
    return phik(k)*(res1 - res2)


def kernel_est_k(X, Y, k, phik=lambda x: 1):
    return torch.sum(torch.tensor([phik(_ki)*Lambda_k(_x, _x, _ki) for _ki in range(k)]))

def mmd_est_k(X, Y, k, phik=lambda x: 1):
    return torch.sum(torch.tensor([M_k(X, Y, ki, phik) for ki in range(k)]), dtype=torch.float64)

In [ ]:
# Test the functions 
_x = h0_paths[0, :, :].unsqueeze(0)
max_k = 10

true_kernel = signature_kernel.compute_kernel(_x, _x)
lambdak_est = kernel_est_k(_x, _x, max_k)

print(f"True kernel value: {true_kernel.item():.4f}. Lambda_k estimate (lvl {max_k}): {lambdak_est.item():.4f}")

In [ ]:
# Test MMD estimator
torch.cuda.empty_cache()
n_paths = 32

testX = h0_paths[:n_paths]
testY = h1_paths[:n_paths]

true_mmd = signature_kernel.compute_mmd(testX, testY)

print(f"True MMD: {true_mmd:.4f}")

In [ ]:
estimate_mmd = mmd_est_k(testX, testY, max_k)

print(f"Estimate MMD: {estimate_mmd:.4f}")

In [ ]:
# Example plot 
_conts = np.array([M_k(testX, testY, _k) for _k in range(max_k)]).cumsum()

plt.figure(figsize=golden_dimensions(4))

#plt.title(f"Unbiased sample MMD vs level-$k$ estimate MMD convergence ($N={n_paths}$)", fontsize="small")
n_bins = int(n_atoms/8)
plt.axhline(true_mmd.cpu(), linestyle="dashed", color="tomato", alpha=0.75, label=r"$\mathcal{D}_{k_{\mathrm{Sig}}}(\mathbf{X}, \mathbf{Y})^2$")
plt.plot(np.arange(max_k), _conts, color="dodgerblue", alpha=0.5, label=r"$\sum_{k> 0}^n M_k(\mathbf{X}, \mathbf{Y})$")
plt.legend()
make_grid()
# plt.savefig("convergence_to_true_phik_1.png", dpi=300)

## 6. Study the differences in signature terms 

The main idea presented here is that, in some cases, the signature terms that differ the most tend to come from the higher orders of the signature. Thus, to minimise the type II error, these terms will need to be more pronounced in the MMD calculation than those of lower order. We first study the different in distribution between these terms to gain a qualitative understanding of how they differ.

In [ ]:
max_k = 5

h0_cont_dists = torch.zeros((max_k, n_atoms))
h1_cont_dists = torch.zeros((max_k, n_atoms))

rand_ints = torch.randint(0, batch_size, size=(n_atoms, 3, n_paths))

for i, ii in enumerate(tqdm(rand_ints)):
    # Sample some paths
    
    x1, x2 = h0_paths[ii[0]], h0_paths[ii[1]]
    y      = h1_paths[ii[-1]]
    
    for j, _k in enumerate(range(1, max_k + 1)):  # 0 order is not interesting
        # Calculate level-k contribution
        h0_cont_dists[j, i] = M_k(x1, x2, _k)
        h1_cont_dists[j, i] = M_k(x1, y, _k)

In [ ]:
# First plot side-by-side
fig, ax = plt.subplots(1, max_k, figsize=(max_k*5, 5))

n_bins = int(n_atoms/8)
for i, axi in enumerate(ax):
    axi.hist(h0_cont_dists[i], bins = n_bins, color="dodgerblue", alpha=0.5, label=
             f"h0", density=True)
    axi.hist(h1_cont_dists[i], bins = n_bins, color="tomato"    , alpha=0.5, label=f"h1", density=True)
    make_grid(axis=axi)
    axi.legend(fontsize="small")
    axi.set_title(f"Level {i+1}")
#fig.suptitle("Distribution of level contributions");
plt.savefig("levelcontributionsbase.png", dpi=300)

It is clear that we don't start to get significant deviation in the distribution of the level contributions under $H_1$ until the level is quite high. However, by then, as from the previous graph, this is only adding a very marginal amount to the value of the MMD. 

We can illustrate this with a separate graph:

In [ ]:
# Define the positions of each histogram
n_bins = int(n_atoms/8)

x_positions = np.arange(max_k+1)[::-1]

plot_dat        = np.zeros((2, max_k + 1, n_atoms))

plot_dat[0, 0]  = h0_dists
plot_dat[0, 1:] = h0_cont_dists

plot_dat[1, 0]  = h1_dists
plot_dat[1, 1:] = h1_cont_dists

# Plot the histograms
colors= ["dodgerblue", "tomato"]

fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

for j, dat in enumerate(plot_dat):
    for i, dat_ in enumerate(dat):
        hist, bin_edges = np.histogram(dat_, bins=n_bins, density=False)
        bin_centers     = (bin_edges[:-1] + bin_edges[1:]) / 2
        width = np.diff(bin_centers)[0]
        
        axes[j].bar(bin_centers, hist, zs=x_positions[i], zdir='y', alpha=0.8, width=width, color=colors[j])
        axes[j].set_xlabel("$M_k$")
        
        yticks      = x_positions + 1
        yticklabels = ["$D(P,Q)^2$"] + [f"level-{k}" for k in range(1, max_k+1)]
        
        axes[j].set_yticks(yticks)
        axes[j].set_yticklabels(yticklabels)


fig.suptitle("Contribution of signature terms, null and alternate hypotheses");
# plt.savefig("levelcontributionhistsbase.png", dpi=300)

## 7. Scaling to minimise the type II error

We now want to see if we can find a scaling such that the contributions from the higher-order terms matter more.

### 7.1 Removing factorial decay: $\phi(k) = (k/2)!$ 

As suggested, we can try to remove the factorial decay by choosing $\pi \sim \text{Exp}(1)$, which has moment function $$\phi(k) = \int \pi^k\, d\mu(\pi) = \int x^k e^{-x}\, dx = \Gamma(k+1) = k!,$$

so (in the PDE setting) in order to arrive at the $\phi$-kernel, we need to do 

\begin{equation*}
    k_\phi(X, Y) \approxeq \frac{1}{M}\sum_{i=1}^M k(\pi^{1/2}X, Y), 
\end{equation*}
where $\pi \sim \text{Exp}(1)$. 

In [ ]:
class PhiKernel(torch.nn.Module):
    def __init__(self, dyadic_order, n_scalings, max_batch=32):
        super().__init__()
        self.sigkernel  = sigkernel.SigKernel(static_kernel=sigkernel.LinearKernel(), dyadic_order=dyadic_order)
        self.scalings   = torch.zeros(n_scalings).exponential_()
        self.n_scalings = n_scalings
        self.max_batch  = max_batch
        
    def forward(self, X, Y):
        res = 0
        for pi in self.scalings:
            res += self.sigkernel.compute_kernel(torch.sqrt(pi)*X, Y, max_batch=self.max_batch)
            
        return res/self.n_scalings
    
    def compute_mmd(self, X, Y):
        res = 0
        for pi in self.scalings:
            K_XX = self.sigkernel.compute_Gram(torch.sqrt(pi)*X, X, sym=False, max_batch=self.max_batch)
            K_XY = self.sigkernel.compute_Gram(torch.sqrt(pi)*X, Y, sym=False, max_batch=self.max_batch)
            K_YY = self.sigkernel.compute_Gram(torch.sqrt(pi)*Y, Y, sym=False, max_batch=self.max_batch)
            
            K_XX_m = (torch.sum(K_XX) - torch.sum(torch.diag(K_XX))) / (K_XX.shape[0] * (K_XX.shape[0] - 1.))
            K_YY_m = (torch.sum(K_YY) - torch.sum(torch.diag(K_YY))) / (K_YY.shape[0] * (K_YY.shape[0] - 1.))
            
            res += K_XX_m + K_YY_m - 2. * torch.mean(K_XY)
        return res/self.n_scalings

In [ ]:
phi_kernel = PhiKernel(dyadic_order=1, n_scalings=25, max_batch=32)
phik       = lambda x: torch.exp(torch.lgamma(torch.tensor(x/2 + 1)))
estimator  = lambda x, y, tk=10, pk=phik: mmd_est_k(x, y, k=tk, phik=pk)

In [ ]:
fac_h0_dists, fac_h1_dists = return_mmd_distributions(
    h0_paths, 
    h1_paths, 
    estimator,
    n_atoms=n_atoms, 
    n_paths=n_paths
)

In [ ]:
fac_crit_val = fac_h0_dists.sort()[0][int(n_atoms*(1-alpha))]

plt.figure(figsize=golden_dimensions(4))
n_bins = int(n_atoms/8)
plt.title(f"Expected Type II error: {100*expected_type2_error(fac_h1_dists, fac_crit_val):.2f}%, $\phi(k) = (k/2)!$", fontsize="small")
plt.hist(fac_h0_dists, bins=n_bins, color="dodgerblue", alpha=0.5, label="$H_0$", density=True)
plt.hist(fac_h1_dists, bins=n_bins, color="tomato"    , alpha=0.5, label="$H_1$", density=True)
plt.legend()
make_grid()

In [ ]:
# Again let's test to see everything is working 
# Note that it's unlikely the convergence will be "perfect" as we are approximating with the PDE method
# and calculating explicitly with the truncated level-k method. Need larger dyadic order too

scalings_ = np.arange(25, 101)
n_runs    = 25
true_mmds = torch.zeros((scalings_.shape[0], n_runs))

for i, n_scaling in enumerate(tqdm(scalings_)):
    for run in range(n_runs):
        this_kernel     = PhiKernel(dyadic_order=0, n_scalings=n_scaling)
        true_mmds[i, run] = this_kernel.compute_mmd(testX, testY)

final_mmd_est = true_mmds[-1].mean()
print(f"True MMD (estimate, {scalings_[-1]} scalings): {final_mmd_est:.4f}")

In [ ]:
max_k = 10

estimate_mmd = mmd_est_k(testX, testY, max_k, phik=phik)

print(f"Estimate MMD: {estimate_mmd:.4f}")

In [ ]:
mean_mmd = true_mmds.mean(axis=1)
sd = true_mmds.std(axis=1)

fig, ax = plt.subplots(figsize=golden_dimensions(4))
ax.fill_between(scalings_, mean_mmd, mean_mmd+sd, color="dodgerblue", alpha=0.25, label="$\pm 1 \sigma$")
ax.plot(scalings_, mean_mmd, label="mmd_estimate", color="dodgerblue", alpha=0.5)
ax.fill_between(scalings_, mean_mmd, mean_mmd-sd, color="dodgerblue", alpha=0.25)
ax.axhline(estimate_mmd, color="tomato", alpha=0.5, label=f"mmd_trunc_sig_{max_k}")
ax.legend(fontsize="small")
ax.set_xlabel("Number of sampled scalings")
ax.set_ylabel("Value of MMD")
ax.set_title("Number of scalings versus estimate of $\phi$-MMD")

In [ ]:
# Example plot 
_conts = np.array([M_k(testX, testY, _k, phik=phik) for _k in range(max_k)]).cumsum()

plt.figure(figsize=golden_dimensions(4))

plt.title(f"Unbiased sample MMD vs level-$k$ estimate MMD convergence ($N={n_paths}$)", fontsize="small")
n_bins = int(n_atoms/8)
plt.axhline(true_mmds[-1].cpu(), linestyle="dashed", color="tomato", alpha=0.75, label="true_mmd_est")
plt.plot(np.arange(max_k), _conts, color="dodgerblue", alpha=0.5, label="level_k_mmd_est")
plt.legend()
make_grid()

There is clearly some estimation error. It looks like quite a few scalings are needed with the signature kernel to get a true estimate of the corresponding $\phi$-kernel.

In [ ]:
max_k   = 6

fac_h0_cont_dists = torch.zeros((max_k, n_atoms))
fac_h1_cont_dists = torch.zeros((max_k, n_atoms))

rand_ints = torch.randint(0, batch_size, size=(n_atoms, 3, n_paths))

for i, ii in enumerate(tqdm(rand_ints)):
    # Sample some paths
    
    x1, x2 = h0_paths[ii[0]], h0_paths[ii[1]]
    y      = h1_paths[ii[-1]]
    
    for j, _k in enumerate(range(1, max_k + 1)):  # 0 order is not interesting
        # Calculate level-k contribution
        fac_h0_cont_dists[j, i] = M_k(x1, x2, _k, phik=phik)
        fac_h1_cont_dists[j, i] = M_k(x1, y, _k, phik=phik) 

In [ ]:
# First plot side-by-side
fig, ax = plt.subplots(1, max_k, figsize=(max_k*5, 5))

n_bins = int(n_atoms/8)

for i, axi in enumerate(ax):
    axi.hist(fac_h0_cont_dists[i], bins = n_bins, color="dodgerblue", alpha=0.5, label=f"h0", density=True)
    axi.hist(fac_h1_cont_dists[i], bins = n_bins, color="tomato"    , alpha=0.5, label=f"h1", density=True)
    make_grid(axis=axi)
    axi.legend(fontsize="small")
    axi.set_title(f"Level {i+1}")
fig.suptitle("Distribution of level contributions");

In [ ]:
# Define the positions of each histogram
n_bins = int(n_atoms/8)

x_positions = np.arange(max_k+1)[::-1]

plot_dat        = np.zeros((2, max_k + 1, n_atoms))
plot_dat[0, 0]  = fac_h0_dists
plot_dat[1, 0]  = fac_h1_dists
plot_dat[0, 1:] = fac_h0_cont_dists
plot_dat[1, 1:] = fac_h1_cont_dists

# Plot the histograms
colors= ["dodgerblue", "tomato"]

fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

for j, dat in enumerate(plot_dat):
    
    for i, dat_ in enumerate(dat):
        hist, bin_edges = np.histogram(dat_, bins=n_bins, density=False)
        bin_centers     = (bin_edges[:-1] + bin_edges[1:]) / 2
        width = np.diff(bin_centers)[0]
        
        axes[j].bar(bin_centers, hist, zs=x_positions[i], zdir='y', alpha=0.8, width=width, color=colors[j])
        axes[j].set_xlabel("$M_k$")
        
        yticks      = x_positions + 1
        yticklabels = ["$D(P,Q)^2$"] + [f"level-{k}" for k in range(1, max_k+1)]
        
        axes[j].set_yticks(yticks)
        axes[j].set_yticklabels(yticklabels)


fig.suptitle("Contribution of signature terms, null and alternate hypotheses, $\phi(k) = (k/2)!$");

### 7.3 Other scaling options: 

#### A "guesstimate": $\phi(k) = dt^{-k/2}$

What seems to work (in practice at least) is a scaling more like $\phi(k) = dt^{-k/2}$. We are working on a more rigourous statement to try and find the correct scaling.

It also may/may not help to normalise the time channel: we do so here, but can be easily remidied. This is a \emph{temporal based approach} that might leverage \emph{annualizing} the stochastic process.

In [ ]:
def batch_path_lengths(paths):
    """
    Computes the lengths of a batch of piecewise linear paths in R^d.
    
    Args:
    paths (Tensor): a tensor of shape (N, l, d) representing N paths, each with l points in R^d.

    Returns:
    Tensor: a tensor of shape (N,) representing the lengths of the paths.
    """
    # Compute the differences between successive points on each path
    diff = paths[:, 1:] - paths[:, :-1]
    
    # Compute the Euclidean norm of the differences
    diff_norm = torch.norm(diff, dim=2)
    
    # Sum up the norms of the differences to get the total length of each path
    lengths = torch.sum(diff_norm, dim=1)
    
    return lengths

In [ ]:
dt = torch.diff(ts)[0]
average_path_length  = torch.mean(batch_path_lengths(h0_paths))
average_state_length = (average_path_length - 1)/state_size

#_scale = torch.pow(dt, -0.5)
_scale = 1

static_kernel = sigkernel.RBFKernel(sigma=1e-2)
signature_kernel = sigkernel.SigKernel(static_kernel=static_kernel, dyadic_order=2)
max_batch = 16
estimator = lambda x, y: signature_kernel.compute_mmd(x, y, max_batch=max_batch)

time_normalisation = True

h0_scaled = _scale*h0_paths.clone()
h1_scaled = _scale*h1_paths.clone()

if time_normalisation:
    h0_scaled[..., 0]  /= _scale*T
    h1_scaled[...,  0] /= _scale*T

In [ ]:
scale_h0_dists, scale_h1_dists = return_mmd_distributions(
    h0_scaled, 
    h1_scaled, 
    estimator, 
    n_atoms=n_atoms, 
    n_paths=n_paths
)

In [ ]:
sc_crit_val = scale_h0_dists.sort()[0][int(n_atoms*(1-alpha))]

fig, ax = plt.subplots(figsize=golden_dimensions(4))
n_bins = int(n_atoms/8)
fig.suptitle(f"Expected Type II error: {100*expected_type2_error(scale_h1_dists, sc_crit_val):.2f}%, $\phi(k) = {_scale:.2f}^k$", fontsize="small")
ax.hist(scale_h0_dists/scale_h0_dists.max(), bins=n_bins, color="dodgerblue", alpha=0.5, label="$H_0$", density=True)
ax.hist(scale_h1_dists/scale_h0_dists.max(), bins=n_bins, color="tomato"    , alpha=0.5, label="$H_1$", density=True)
ax.legend()
make_grid(axis=ax)
# plt.savefig("workedexampledistsdtsq.png", dpi=300)

In [ ]:
max_k_sc = 10

sc_testX = h0_scaled[:n_paths]
sc_testY = h1_scaled[:n_paths]

true_mmd_sc = signature_kernel.compute_mmd(sc_testX, sc_testY, max_batch=16)

print(f"True MMD: {true_mmd_sc:.4f}")

In [ ]:
estimate_mmd = mmd_est_k(sc_testX, sc_testY, max_k_sc)

print(f"Estimate MMD: {estimate_mmd:.4f}")

In [ ]:
_conts = np.array([M_k(sc_testX, sc_testY, _k) for _k in range(max_k_sc)]).cumsum()

plt.figure(figsize=golden_dimensions(4))
#plt.title(f"Unbiased sample MMD vs level-$k$ estimate MMD convergence ($N={n_paths}$)", fontsize="small")
n_bins = int(n_atoms/8)
plt.axhline(true_mmd_sc.cpu(), linestyle="dashed", color="tomato", alpha=0.75, label=r"$\mathcal{D}_{k_{\mathrm{Sig}}}(\mathbf{X}, \mathbf{Y})^2$")
plt.plot(np.arange(max_k_sc), _conts, color="dodgerblue", alpha=0.5, label=r"$\sum_{k> 0}^n M_k(\mathbf{X}, \mathbf{Y})$")
plt.legend()
# make_grid()
# plt.savefig("convergence_to_true_phik_dtsq.png", dpi=300)

Convergence now happens much later: compared to the base case, at around $k=10$, compared to around $k=3$ in the unscaled case.

In [ ]:
max_k   = 5

sc_h0_cont_dists = torch.zeros((max_k, n_atoms))
sc_h1_cont_dists = torch.zeros((max_k, n_atoms))

rand_ints = torch.randint(0, batch_size, size=(n_atoms, 3, n_paths))

for i, ii in enumerate(tqdm(rand_ints)):
    # Sample some paths
    
    x1, x2 = h0_scaled[ii[0]], h0_scaled[ii[1]]
    y      = h1_scaled[ii[-1]]
    
    for j, _k in enumerate(range(1, max_k + 1)):  # 0 order is not interesting
        # Calculate level-k contribution
        sc_h0_cont_dists[j, i] = M_k(x1, x2, _k)
        sc_h1_cont_dists[j, i] = M_k(x1, y, _k)

In [ ]:
# First plot side-by-side

fig, ax = plt.subplots(1, max_k, figsize=(max_k*5, 5))

n_bins = int(n_atoms/8)

for i, axi in enumerate(ax):
    axi.hist(sc_h0_cont_dists[i], bins = n_bins, color="dodgerblue", alpha=0.5, label=f"h0", density=True)
    axi.hist(sc_h1_cont_dists[i], bins = n_bins, color="tomato"    , alpha=0.5, label=f"h1", density=True)
    make_grid(axis=axi)
    axi.legend(fontsize="small")
    axi.set_title(f"Level {i+1}")
fig.suptitle("Distribution of level contributions, scaled");
# plt.savefig("levelcontributionsdtsq.png", dpi=300)

The difference is clear.

In [ ]:
# Define the positions of each histogram
n_bins = int(n_atoms/8)

x_positions = np.arange(max_k+1)[::-1]

plot_dat        = np.zeros((2, max_k + 1, n_atoms))
plot_dat[0, 0]  = scale_h0_dists
plot_dat[1, 0]  = scale_h1_dists
plot_dat[0, 1:] = sc_h0_cont_dists
plot_dat[1, 1:] = sc_h1_cont_dists

# Plot the histograms
colors= ["dodgerblue", "tomato"]

fig, axes = plt.subplots(1, 2, figsize=(12, 6), subplot_kw={'projection': '3d'})

for j, dat in enumerate(plot_dat):
    
    for i, dat_ in enumerate(dat):
        hist, bin_edges = np.histogram(dat_, bins=n_bins, density=False)
        bin_centers     = (bin_edges[:-1] + bin_edges[1:]) / 2
        width = np.diff(bin_centers)[0]
        
        axes[j].bar(bin_centers, hist, zs=x_positions[i], zdir='y', alpha=0.8, width=width, color=colors[j])
        axes[j].set_xlabel("$M_k$")
        
        yticks      = x_positions + 1
        yticklabels = ["$D(P,Q)^2$"] + [f"level-{k}" for k in range(1, max_k+1)]
        
        axes[j].set_yticks(yticks)
        axes[j].set_yticklabels(yticklabels)


elev = 30 # Example elevation
azim = -60 # Example azimuth

axes[0].view_init(elev=elev, azim=azim)
axes[1].view_init(elev=elev, azim=azim)

fig.suptitle("Contribution of signature terms, null and alternate hypotheses, $\phi(k) = dt^{-k/2}$");
# plt.savefig("levelcontributionhistsdtsq.png", dpi=300)

Greater contributions occur much later - as desired.

#### Relative density of letters in level $k$

Consider time-augmented paths $X: [a,b] \to \mathbb{R}$. Suppose you are interested in having data like the $k^{\text{th}}$ moment be the most relavant in the calculation of the signature kernel MMD. 

##### Notation

1. $\mathcal{A}_d$: alphabet of letters $\{1, 2, \dots, d\}$. 
2. $\mathcal{W}(\mathcal{A}_d)$: Space of words comprised of letters from $\mathcal{A}_d$ 
3. $\mathcal{W}^N(\mathcal{A}_d) \subset \mathcal{W}(\mathcal{A}_d)$: Space of words of length $N \in \mathbb{N}$
4. $\xi: \mathcal{W}(\mathcal{A}_d) \times \mathcal{A}_d \to \mathbb{N}$: function that returns the number of times the letter $i$ appears in the word $\mathbf{w}$.

Here we fix $d=2$ (time-augmented univariate process). 

<b>Hypothesis</b>: the level $k$ that contains the highest <i>relative density</i> of terms like $$S(X)_{[a,b]}^{I^k_N}, \quad I^k_N \in \{\xi(I_N, i) = k : I_N \in \mathcal{W}^N(\mathcal{A}_2) \}$$ is the level which should contribute maximally to the MMD. 

We define the following terms (for general $d \in \mathbb{N}$):

1. The $k$-<i>incidence</i> of the letter $i$ at the level $N$ is given by $$\iota^N(i, k) = \sum_{w \in \mathcal{W}^N(\mathcal{A}_d)} \chi_{\{\xi(w, i) = k\}}(w).$$ When $d=2$, $\iota^N(i, k) = {N \choose k}$ where $N$ is the length of the word $\boldsymbol{w}$.

2. The $k$-<i>density</i> of the letter $i$ at the level $N$ is given by $$\delta^N(i, k) = \frac{\iota^N(i, k)}{|\mathcal{W}^N(\mathcal{A}_d)|}$$ where $|\cdot|$ denotes the number of words that can be made with $N$ letters. In fact, it is given by $$|\mathcal{W}^N(\mathcal{A}_d)| = d^N.$$ When $d=2$, this reduces to $$\delta^N(i, k) = \frac{{N \choose k}}{2^N} = \frac{N!}{2^Nk!(N-k)!}.$$

Some notes: 

1. The $k$-incidence is an increasing function in $N$ (obvious: for $d > 1$, all terms contributing to the $k$-incidence at level $N$ also contribute at level $N+1$ when integrated against channels not corresponding to $i$).

2. <b>Hypothesis</b>: There exists a (not necessarily unique) maximal $N\in \mathbb{N}$ such that the $k$-density of a letter $i$ is maximised.

3. <b>Question</b>: Does the function $N \mapsto \delta^N(i, k)$ have a maximum?

##### Method

We stick to the time-augmented univariate case here.

1. Find the $N \in \mathbb{N}$ that maximises the $k$-density of the letter $2$.
2. Find the scaling $\lambda \in \mathbb{R}$ that makes the term like $L(\lambda x)^N/N!$ the largest. 
3. Apply this scaling; check the results.

This technique can be easily extended to a distribution scalings, representing emphasis weights on certain moments. One then needs to solve the appropriate Hamburger moment problem.

In [ ]:
# First, let's check all the assumptions about n choose k and so on hold.
def nchoosek(N, k):
    return math.factorial(N)/(math.factorial(k)*math.factorial(N-k))

def generate_words(N, d):
    alphabet = torch.arange(d)  # Create the alphabet as a tensor from 0 to d-1
    words = torch.zeros((d**N, N), dtype=torch.long)  # Initialize a tensor to store the generated words

    for i in range(N):
        repeat = d**(N-i-1)  # Number of times to repeat each letter
        tile   = d**i        # Number of times to tile the repeated letters

        letters = alphabet.repeat(repeat)  # Repeat each letter
        letters = letters.unsqueeze(1).repeat(1, tile).view(-1)  # Tile the repeated letters

        words[:, i] = letters  # Assign the letters to the corresponding column of the words tensor

    return words
    

def k_incidence(N, d, k, i):
    """
    Returns the number of words of length :param N: from the alphabet of :param d: letters 
    that contain the letter :param i: :param k: times
    """
    res = 0
    words = generate_words(N, d)
    
    for w in words:
        letter_freq = sum(i == w)
        res += letter_freq == k
        
    return res
    
def k_density(N, d, k, i):
    return k_incidence(N, d, k, i)/torch.pow(d, N)

In [ ]:
# Example with the second moment

d = torch.tensor(state_size + 1)  # Number of dimensions in asset price process
i = 1                             # Stock price channel 
k = 2                             # Second moment 
max_N = 12                        # An example

N_range = torch.arange(1, max_N + 1)
d_range = torch.pow(d, N_range)

k_incidences = torch.tensor([k_incidence(Ni, d, k, i) for Ni in N_range])
k_densities  = k_incidences/d_range

fig, ax = plt.subplots(figsize=golden_dimensions(4))
ax.plot(k_incidences, color="dodgerblue", label="$\iota^N(k, 1)$", alpha=0.5)
ax2 = ax.twinx()
ax2.plot(k_densities, color="tomato", label="$\delta^N(k, 1)$", alpha=0.5)
handles1, labels1 = ax.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()

# Combine handles and labels
handles = handles1 + handles2
labels  = labels1 + labels2

# Create the combined legend
ax.legend(handles, labels, loc=1)

In [ ]:
# Here it appears the second/third level should matter the most 
# We will scale so that the maximum contribution comes from these values
# We now plot the bounds function associated to the h0_paths 

average_path_length = torch.mean(batch_path_lengths(h0_paths))
bounds = torch.tensor([torch.pow(average_path_length, k)/math.factorial(k) for k in N_range])
plt.plot(bounds, color="dodgerblue", alpha=0.5)